In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [2]:
OPT_INDEX = '/Lun0/zhiyong/libri'
# data_dir = '/Lun0/zhiyong/dataset/vox2/dev/aac'
# data_dir_1 = '/Lun0/zhiyong/dataset/vox1/dev/wav'
# data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'
data_dir_100 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100'
data_dir_360 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360'
data_dir_o500 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500'
new_start_id = 7911 #vox with 0-7322 in with 7323-7910


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
# val_utt_num = 200

In [18]:
spk2utt = {}
# spk2utt_train = {}
# spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(data_dir_100+'/*')
label = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_360+'/*')
label_1 = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_o500+'/*')
label_2 = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = glob.glob(os.path.join(data_dir_100, i)+'/*/*.flac')

for count_1, i in enumerate(label_1):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1] = glob.glob(os.path.join(data_dir_360, i)+'/*/*.flac')

for count_2, i in enumerate(label_2):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1+1+count_2)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1+1+count_2] = glob.glob(os.path.join(data_dir_o500, i)+'/*/*.flac')

In [4]:
# should be class:
251+921+1166

2338

In [7]:
7910 + 2338

10248

In [13]:
len(spk2utt)

2338

In [14]:
spk2utt.keys()

dict_keys([7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975, 7976, 7977, 7978, 7979, 7980, 7981, 7982, 7983, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8010, 8011, 8012, 8013, 8014, 8015, 8016, 8017, 8018, 8019, 8020, 8021, 8022, 8023, 8024, 8025, 8026, 8027, 8028, 8029, 8030, 8031, 8032, 8033, 8034, 8035, 8036, 8037, 8038, 8039, 8040, 8041, 8042, 8043, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8054, 8055, 8056, 8057, 8058, 8059, 8060, 8061, 8062, 8063, 8064, 8065, 8066, 8067, 8068, 8069, 8070, 8071, 8072, 8073, 8074, 8075,

In [19]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [22]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)

In [23]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [24]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [25]:
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# spk2utt_val_len = os.path.join(OPT_INDEX, 'spk2utt_val_len')

spk2utt_train_len_dict = {}
# spk2utt_val_len_dict = {}

In [26]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [49]:
# trial dict
trial_dir = '/Lun0/zhiyong/dataset/vox1/test/wav'
trial_dict_out = os.path.join(OPT_INDEX, 'trial_dict')
trial_dict = {}

tmp_dir_list = glob.glob(trial_dir+'/*/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    trial_dict[count] = this_dir
with open(trial_dict_out, 'wb') as handle:
    pickle.dump(trial_dict, handle)

In [52]:
len(trial_dict)

4874

# make small 1172 train

In [4]:
251+921

1172

In [5]:
7323+488

7811

In [7]:
from preprocessing import VoxIterableDataset

In [8]:
OPT_INDEX = '/Lun0/zhiyong/libri'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'
data_dir_dict = {}

# train
# data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [9]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [10]:
with open(os.path.join(OPT_INDEX_2, 'spk2utt_train_dict'), 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [11]:
spk2utt_train_dict.keys()

dict_keys([7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975, 7976, 7977, 7978, 7979, 7980, 7981, 7982, 7983, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8010, 8011, 8012, 8013, 8014, 8015, 8016, 8017, 8018, 8019, 8020, 8021, 8022, 8023, 8024, 8025, 8026, 8027, 8028, 8029, 8030, 8031, 8032, 8033, 8034, 8035, 8036, 8037, 8038, 8039, 8040, 8041, 8042, 8043, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8054, 8055, 8056, 8057, 8058, 8059, 8060, 8061, 8062, 8063, 8064, 8065, 8066, 8067, 8068, 8069, 8070, 8071, 8072, 8073, 8074, 8075,

In [12]:
spk2utt_train_dict_small_1172 = {}
spk2utt_val_dict_small_1172 = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 1172:
        spk2utt_train_dict_small_1172[7323+488+count] = spk2utt_train_dict[i]
    else:
        break

In [13]:
spk2utt_train_dict_small_1172.keys()

dict_keys([7811, 7812, 7813, 7814, 7815, 7816, 7817, 7818, 7819, 7820, 7821, 7822, 7823, 7824, 7825, 7826, 7827, 7828, 7829, 7830, 7831, 7832, 7833, 7834, 7835, 7836, 7837, 7838, 7839, 7840, 7841, 7842, 7843, 7844, 7845, 7846, 7847, 7848, 7849, 7850, 7851, 7852, 7853, 7854, 7855, 7856, 7857, 7858, 7859, 7860, 7861, 7862, 7863, 7864, 7865, 7866, 7867, 7868, 7869, 7870, 7871, 7872, 7873, 7874, 7875, 7876, 7877, 7878, 7879, 7880, 7881, 7882, 7883, 7884, 7885, 7886, 7887, 7888, 7889, 7890, 7891, 7892, 7893, 7894, 7895, 7896, 7897, 7898, 7899, 7900, 7901, 7902, 7903, 7904, 7905, 7906, 7907, 7908, 7909, 7910, 7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975,

In [15]:
spk2utt_train_dict_small_1172[7811]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0039.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0029.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0023.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0011.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0020.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0031.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0019.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0008.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0000.flac',
 

In [16]:
for i in spk2utt_train_dict_small_1172:
    spk2utt_val_dict_small_1172[i] = [spk2utt_train_dict_small_1172[i][0]]
    spk2utt_train_dict_small_1172[i].pop(0)

In [18]:
spk2utt_train_dict_small_1172[7811]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0029.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0023.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0011.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0020.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0031.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0019.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0008.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0000.flac',
 '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0016.flac',
 

In [19]:
spk2utt_val_dict_small_1172[7811]

['/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/6563/285357/6563-285357-0039.flac']

In [21]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_full_1172'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small_1172, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_dict_full_1172'), 'wb') as handle:
    pickle.dump(spk2utt_val_dict_small_1172, handle)

In [27]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_full_1172')

In [25]:
1500 * 1172

1758000

In [28]:
config = {}

config['sr'] = 16000
config['repeats'] = 150
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/libri/train_data_full_1172/'
config['err_log'] = '/Lun0/zhiyong/libri/tmp_data_errlog'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open(config['err_log'], 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(10)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/882/123268/882-123268-0027.flac 8.306875 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/8119/115687/8119-115687-0052.flac 1.3351875 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.76775 4.99
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.4945625 4.99
4
4
4
4
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.366 4.99
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
6
6
6
6
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/4788/94904/4788-94904-0018.flac 1.276 4.99
6
6
6
6
6
6
7
7
7
7
8
8
8
7
7
7
8
7
7


# Make train index

In [29]:
1500 * 1172

1758000

In [31]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_full_1172')
expected_len = 1758000
workers = 10
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_full_1172.csv')

In [32]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [33]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [34]:
count

1758000

# Make val data

In [35]:
OPT_INDEX = '/Lun0/zhiyong/libri'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
OPT_INDEX_2 = '/Lun0/zhiyong/libri'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict_full_1172')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [36]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX_2, 'spk2utt_train_len')

In [38]:
config = {}

config['sr'] = 16000
config['repeats'] = 1
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/libri/val_data_full_1172/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.07797026634216309
0.049703359603881836
0.042147159576416016
0.0323483943939209
0.021860599517822266
0.04619956016540527
0.028306007385253906
0.045069217681884766
0.030568361282348633
0.024224042892456055
0.0580592155456543
0.050839900970458984
0.026983022689819336
0.030658245086669922
0.02724599838256836
0.026591777801513672
0.05547189712524414
0.02545762062072754
0.05128288269042969
0.027809858322143555
0.06516003608703613
0.05508685111999512
0.021835803985595703
0.032686710357666016
0.02746415138244629
0.04717826843261719
0.021551132202148438
0.02796459197998047
0.03997659683227539
0.06052875518798828
0.029755592346191406
0.0444638729095459
0.02294135093688965
0.045725107192993164
0.05200648307800293
0.044367074966430664
0.021210908889770508
0.043547868728637695
0.057955026626586914
0.03881478309631348
0.02000260353088379
0.02842926979064941

0.028487205505371094
0.040177345275878906
0.02373814582824707
0.028360605239868164
0.03363966941833496
0.038196563720703125
0.028542757034301758
0.0296933650970459
0.02469611167907715
0.022298812866210938
0.03563284873962402
0.023832082748413086
0.0483403205871582
0.044567108154296875
0.04269695281982422
0.022216320037841797
0.028013944625854492
0.02406597137451172
0.029630422592163086
0.024065256118774414
0.04702115058898926
0.040343284606933594
0.028643369674682617
0.021925926208496094
0.027760982513427734
0.04020380973815918
0.04401731491088867
0.03955698013305664
0.022100210189819336
0.02457261085510254
0.03112959861755371
0.03049182891845703
0.029022216796875
0.028768062591552734
0.030245542526245117
0.04586291313171387
0.04491138458251953
0.02156805992126465
0.02195906639099121
0.0499722957611084
0.05313706398010254
0.05310535430908203
0.021284103393554688
0.0394134521484375
0.02884984016418457
0.029073238372802734
0.023862600326538086
0.04481196403503418
0.02540111541748047
0.03

0.022395849227905273
0.026146888732910156
0.02475881576538086
0.04821419715881348
0.025107622146606445
0.05180239677429199
0.035164833068847656
0.026474952697753906
0.024682283401489258
0.05067014694213867
0.04725289344787598
0.04614543914794922
0.02576470375061035
0.03297138214111328
0.022693395614624023
0.025274038314819336
0.03115367889404297
0.025716066360473633
0.03365826606750488
0.026702165603637695
0.026136159896850586
0.03493356704711914
0.036536216735839844
0.0458376407623291
0.04410672187805176
0.04794597625732422
0.045809030532836914
0.026233196258544922
0.04557490348815918
0.03993821144104004
0.02036285400390625
0.042037010192871094
0.031925201416015625
0.027100563049316406
0.0327913761138916
0.054913997650146484
0.028313636779785156
0.03433990478515625
0.04601573944091797
0.022565126419067383
0.04560208320617676
0.022858619689941406
0.02779245376586914
0.026206493377685547
0.024132728576660156
0.026802778244018555
0.03540539741516113
0.05216717720031738
0.0293564796447753

# Make validation index (Need only once)

In [39]:
train_data_dir = os.path.join(OPT_INDEX, 'val_data_full_1172')
expected_len = 1172 * 1
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_full_1172.csv')

In [40]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [41]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [42]:
count

1172